# Классификация клиентов телеком компании

Оператор мобильной связи выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В нашем распоряжении уже предобработанные данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф.

**Цель проекта:**  
Построить модель классификации с максимально большим значением *accuracy* (необходимо довести долю правильных ответов по крайней мере до 0.75 и проверить *accuracy* на тестовой выборке).


**Ход исследования:**
 * Шаг 1: Изучение данных из файла
 * Шаг 2: Разбиение данных на выборки
 * Шаг 3: Исследование моделей
 * Шаг 4: Проверка моделей на тестовой выборке
 * Шаг 5: Проверка моделей на адекватность
 * Шаг 6: Выводы

In [1]:
# импорт необходимых библиотек
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

from sklearn.metrics import accuracy_score

## Изучение данных из файла

Прочитаем файл и сохраним его в переменной `df`

Для ознакомления с данными выведем на экран информацию о таблице и первые 10 строк таблицы:

In [3]:
print(df.info())
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


Каждый из 3214 объектов в наборе данных — это информация о поведении одного пользователя за месяц.  
Согласно документации к данным:
* `сalls` — количество звонков,
* `minutes` — суммарная длительность звонков в минутах,
* `messages` — количество sms-сообщений,
* `mb_used` — израсходованный интернет-трафик в Мб,
* `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Данные предварительно обработаны и не содержат пропусков. Тем не менее, кратко проверим таблицу на наличие дубликатов, а также выведем минимальные и максимальные значения каждого столбца, чтобы убедиться в отсутствии аномалий - отрицательных значений.

In [4]:
# подсчёт явных дубликатов
print('Duplicates:', df.duplicated().sum(), '\n')
# вывод минимума и максимума столбца
for column in df.columns:
    print(column, '\nMin:', int(df[column].min()), '\nMax:', int(df[column].max()), '\n')
print(df['is_ultra'].unique())

Duplicates: 0 

calls 
Min: 0 
Max: 244 

minutes 
Min: 0 
Max: 1632 

messages 
Min: 0 
Max: 224 

mb_used 
Min: 0 
Max: 49745 

is_ultra 
Min: 0 
Max: 1 

[0 1]


Итак, в нашем распоряжении 3214 строк, содержащих информацию о количестве израсходованных минут, сообщений и Мб интернет-трафика для каждого пользователя в месяц. Данные предварительно обработаны и не содержат пропусков, дубликатов и аномалий.

С помощью предоставленных данных необходимо посторить модель классификации, позволяющую на основании количества использованных услуг предложить клиенту оптимальный тариф.

## Разбиение данных на выборки

Извлечем из таблицы признаки и целевой признак:

In [5]:
features = df.drop('is_ultra',  axis=1) # извлекаем признаки
target = df['is_ultra']  # извлекаем целевой признак

print(features.shape)
target.shape

(3214, 4)


(3214,)

Извлечение целевого признака произошло успешно, далее данные необходимо разбить на три части: обучающую, валидационную и тестовую. Размеры тестового и валидационного наборов обычно равны и исходные данные разбивают в соотношении 3:1:1.

In [6]:
# разделим выборку на две части: обучающую и тестовую
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=.4, random_state=0)
# разделим тестовую часть пополам - выделим валидационную часть
features_valid, features_test, target_valid, target_test = train_test_split(
    features_test, target_test, test_size=.5, random_state=0)

Проверим, что выборки разделились корректно: размеры тестовой и валидационной выборки должны быть равны.

In [7]:
for matrix in [features_train, features_valid, features_test]:
    print(matrix.shape)

(1928, 4)
(643, 4)
(643, 4)


In [8]:
for vector in [target_train, target_valid, target_test]:
    print(vector.shape)

(1928,)
(643,)
(643,)


Итак, разделение выборок на обучающую, валидационную и тестовую произошло успешно, можно приступать к обучению моделей.

## Исследование моделей

Применим к подготовленным данным три различные модели обучения для решения задачи классификации: решающее дерево, решающий лес и логистическую регрессию. Для решающего леса и решающего дерева рассмотрим несколько значений гиперпараметров и выберем наиболее оптимальные из них.

### Решающее дерево

Рассмотрим модель решающего дерева при разных значениях глубины дерева (от 1 до 10) и выберем глубину, позволяющую достичь максимального значения доли правильных предсказаний модели.

In [9]:
best_model_tree = None
best_accuracy_tree = 0

for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=0, max_depth=depth) # обучаем модель с заданной глубиной дерева
    model.fit(features_train, target_train) # обучаем модель на обучающей выборке
    predictions_valid = model.predict(features_valid) # получим предсказания модели на валидационной выборке
    accuracy = accuracy_score(target_valid, predictions_valid) # посчитаем качество модели на валидационной выборке
    
    if accuracy > best_accuracy_tree:
        best_model_tree = model
        best_accuracy_tree = accuracy
        
print("Accuracy лучшей модели решающего дерева:", best_accuracy_tree.round(3), "\nМаксимальная глубина:", depth)

Accuracy лучшей модели решающего дерева: 0.812 
Максимальная глубина: 10


С помощью модели решающего дерева с глубиной 10 удалось достичь значения accuracy, равного 81%. Таким образом, выбранная модель решающего дерева не переобучается и выдаёт приемлемое значение доли правильных ответов.  

### Решающий лес

Рассмотрим следующую модель - решающий лес. Будем менять количество деревьев от 10 до 50 с шагом в 5 деревьев, а также максимальную глубину этих деревьев, чтобы определить оптимальные гиперпараметры для получения наибольшего значения точности предсказаний модели.

In [10]:
best_model_forest = None
best_accuracy_forest = 0
best_est = 0
best_depth = 0

for est in range(10, 51, 5): # меняем количество деревьев
    for depth in range (1, 11): # меняем максимальную глубину деревьев
        model = RandomForestClassifier(random_state=0, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train) # обучаем модель на тренировочной выборке
        predictions_valid = model.predict(features_valid) # получаем предсказания модели на валидационной выборке
        accuracy = accuracy_score(target_valid, predictions_valid) # считаем значение accuracy на валидационной выборке
        
        if accuracy > best_accuracy_forest:
            best_model_forest = model
            best_accuracy_forest = accuracy
            best_est = est
            best_depth = depth

print("Accuracy наилучшей модели решающего леса на валидационной выборке:", best_accuracy_forest.round(3), 
      "\nКоличество деревьев:", best_est, "\nМаксимальная глубина:", depth)

Accuracy наилучшей модели решающего леса на валидационной выборке: 0.821 
Количество деревьев: 35 
Максимальная глубина: 10


При количестве деревьев, равному 35, и при максимальной глубине деревьев, равной 10, модель решающего леса достигает точности 82% на валидационной выборке. Аналогично предыдущему случаю, модель не переобучается и достаточно точно предсказывает правильные ответы.

### Логистическая регрессия

Проверим, как с поставленной задачей справится модель логистической регрессии.

In [11]:
model_log_reg = LogisticRegression(random_state=0, solver='lbfgs', max_iter=100) # задаем параметры модели
model_log_reg.fit(features_train, target_train) # обучаем на тренировочной выборке

accuracy_log_reg = model_log_reg.score(features_valid, target_valid) # получим значение точности на валидационной выборке

print("Accuracy модели логистической регрессии на валидационной выборке:", accuracy_log_reg.round(3))

Accuracy модели логистической регрессии на валидационной выборке: 0.743


Модель логистической регрессии справилась с предсказаниями на валидационной выборке хуже предыдущих моделей: точность предсказаний составила 74%.

**Вывод** 

Максимальное значение точности предсказаний на валидационной выборке (82%) удалось получить при применении модели решающего леса с 35 деревьями и максимальной глубиной деревьев, равной 10.
Модель решающего дерева с глубиной 10 позволяет получить точность, равную 81%, а модель логистической регрессии на предоставленных данных справляется хуже: точность на валидацонной выборке равна 74%.

## Проверка модели на тестовой выборке

Проверим модели решающего дерева, решающего леса и логистической регрессии с оптимально подобранными гиперпараметрами на тестовой выборке.  
Предварительно объединим обучающую и валидационную выборки и заново обучим каждую модель на объединённой выборке.

In [12]:
features_train = features_train.append(features_valid)
target_train = target_train.append(target_valid)

print(features_train.shape)
print(target_train.shape)

(2571, 4)
(2571,)


**Проверка модели решающего дерева**

In [13]:
best_model_tree.fit(features_train, target_train)
predictions_test_tree = best_model_tree.predict(features_test)
accuracy_test_tree = accuracy_score(target_test, predictions_test_tree)
print("Accuracy модели решающего дерева на тестовой выборке:", accuracy_test_tree.round(3))

Accuracy модели решающего дерева на тестовой выборке: 0.76


**Проверка модели решающего леса**

In [14]:
best_model_forest.fit(features_train, target_train)
predictions_test_forest = best_model_forest.predict(features_test)
accuracy_test_forest = accuracy_score(target_test, predictions_test_forest)
print("Accuracy модели решающего леса на тестовой выборке:", accuracy_test_forest.round(3))

Accuracy модели решающего леса на тестовой выборке: 0.773


**Проверка модели логистической регрессии**

In [15]:
model_log_reg.fit(features_train, target_train)
accuracy_log_reg_test = model_log_reg.score(features_test, target_test)
print("Accuracy модели логистической регрессии на тестовой выборке:", accuracy_log_reg_test.round(3))

Accuracy модели логистической регрессии на тестовой выборке: 0.751


**Вывод**

Наилучшие результаты на тестовой выборке получаются при использовании моделей решающего леса (точность более 0.77) и решающего дерева (точность составляет 0.76).  
При этом результаты работы модели логистической регрессии на тестовой выборке даже немного лучше, чем на валидационной: значение accuracy превысило 0.75.

## Проверка моделей на адекватность

Сравним работу наилучшей модели решающего леса с максимальным значением accuracy на тестовой выборке с работой наивной модели, которая при любых входных данных возвращает значение преобладающего в обучающей выборке класса.

In [16]:
dummy_clf = DummyClassifier(strategy='most_frequent', random_state=0)
dummy_clf.fit(features_train, target_train)

print( "Accuracy наивной модели:", dummy_clf.score(features_test, target_test).round(3))
print("Accuracy модели решающего леса на тестовой выборке:", accuracy_test_forest.round(3))

Accuracy наивной модели: 0.686
Accuracy модели решающего леса на тестовой выборке: 0.773


Accuracy модели решающего леса с 35 деревьями и максимальной глубиной деревьев, равной 10, превышает accuracy наивной модели: следовательно, наша модель решающего леса вполне адекватно предсказывает тариф пользователя.

## Вывод

Целью проекта является построение для компании "Мегалайн" модели классификации с максимально большим значением accuracy, позволяющей предлагать пользователям компании наиболее подходящий для них тариф ("Смарт" или "Ультра").  
В ходе выполнения задания проекта был произведен обзор данных компании, данные были разделены на обучающую, валидационную и тестовую выборки в соотношении 3:1:1, после чего были исследованы следующие модели при различных значениях гиперпараметров:
* модель решающего дерева
* модель решающего леса
* логистическая регрессия

В результате обучения этих моделей при различных гиперпараметрах и их проверке на валидационных выборках было обнаружено, что:
* максимальное значение точности предсказаний на валидационной выборке удалось получить при применении модели решающего леса с 35 деревьями и максимальной глубиной деревьев, равной 10. Accuracy данной модели равна 0.82
* Модель решающего дерева с глубиной 10 позволяет получить точность, равную 0.81
* модель логистической регрессии на валидационной выборке позволила получить точность 0.74

При проверке моделей на тестовой выборке наилучший результат получился также при применении модели решающего леса (accuracy = 0.77). Модель решающего дерева не сильно уступает по точности: accuracy = 0.76.И, наконец, модель логистической регресси на тестовой выборке позволяет получить значение accuracy, равное 0.75.

Наконец, accuracy лучшей модели решающего леса (с 35 деревьями и максимальной глубиной деревьев, равной 10) превышает accuracy наивной модели (0.77 против 0.69): следовательно, наша модель решающего леса способна предсказывать тариф пользователя вполне разумно.